In [44]:
# 180827 indel_filter script written by J.Youk

import sys, os, pysam

def insertion_bam(input_line, bam_file, ref_fasta):
    input_split = input_line.split('\t')
    input_chr = input_split[0]
    input_pos = long(input_split[1])
    initial_ru = input_split[4][1:]
    initial_rlen = len(initial_ru)
    call_ru = initial_ru
    initial_rc = 0

    for k in range(1,(initial_rlen+1)):
        min_repeat = []
        k_start = 0
        temp_ru = ''
        #print k
        if (initial_rlen % k) !=0:
            continue
        else:
            while k_start < initial_rlen:                
                temp_ru = initial_ru[k_start:k_start+k]
                if temp_ru in min_repeat:
                    'blank'
                else:
                    min_repeat.append(temp_ru)
                    if len(min_repeat) > 1:
                        break
                k_start += k
            if len(min_repeat) == 1:
                call_ru = temp_ru
                call_rlen = len(call_ru)
                initial_rc = initial_rlen / call_rlen
                #print initial_rc
                break
    
    ref_rc = 0
    nref = 0; nvar = 0; nother = 0; ndup = 0
    temp_pos = input_pos
    var_seq =[]; ref_seq1=[]; ref_seq2=[]
    
    while 1:
        if ref_fasta.fetch(input_chr,temp_pos,temp_pos+call_rlen) == call_ru:
            ref_rc += 1
            temp_pos = temp_pos + call_rlen
        else:
            break
        
    repeat_start = input_pos
    repeat_end = input_pos + ref_rc*call_rlen
    #print call_ru, call_rlen, ref_rc, repeat_start, repeat_end
    #print 'repetitive_reference_sequence = ' + ref_fasta.fetch(input_chr,repeat_start, repeat_end)
    temp_num = 0
    temp_seq2 = []
        
    for read in bam_file.fetch(input_chr,input_pos-1,input_pos+1):
        temp_num += 1
        target = ''
        temp_seq = ''
        temp_seq1 = []
        target_length = 0
        if read.cigartuples == None:
            continue
        else:
            #print read
            #print read.reference_start, read.reference_end
            #print read.query_alignment_start, read.query_alignment_end
            for nn in range(0,initial_rc+1):
                if ((read.reference_start - read.query_alignment_start) <= repeat_start - (initial_rc-nn) * call_rlen) and ((read.reference_end + 151 - read.query_alignment_end) >= repeat_end + nn*call_rlen):
                    target_length = 1
                    break
            
            if target_length == 1:
                #print 'target_read'
                abs_pos = read.reference_start # include clip
                rel_pos = read.query_alignment_start # consider left softclip, hardclip (ex> cigar: 10s141M => 10)
                remain_distance = input_pos - abs_pos
                
                cigar_list = read.cigartuples               
                for cigar in cigar_list:
                    if cigar[0] == 0:
                        if (abs_pos + cigar[1]) < input_pos:
                            abs_pos += cigar[1]
                            rel_pos += cigar[1]
                            remain_distance -= cigar[1]
                        elif (abs_pos + cigar[1]) == input_pos:
                            abs_pos += cigar[1]
                            rel_pos += cigar[1]
                            remain_distance -= cigar[1]
                            target = 'no_var'
                        else:
                            rel_pos += remain_distance
                            abs_pos += remain_distance
                            remain_distance = 0
                            target = 'no_var'
                            break
                    elif cigar[0] == 1:
                        if abs_pos == input_pos:
                            if read.query_sequence[rel_pos:(rel_pos + cigar[1])] == initial_ru:
                                target = 'var'
                                break
                            else:
                                target = 'no_var'
                                break
                        else:
                            rel_pos += cigar[1] # no change of abs_pos and remain_distance
                    elif cigar[0] == 2:
                        if abs_pos == input_pos:
                            target = 'no_var'
                            break
                        else:
                            if (abs_pos + cigar[1]) <= input_pos:
                                abs_pos += cigar[1] # no change of rel_pos 
                                remain_distance -= cigar[1]
                            else:
                                abs_pos += remain_distance
                                rel_pos += remain_distance
                                remain_distance = 0
                                target = 'no_var'
                                break
                    elif cigar[0] == 3:
                        if (abs_pos + cigar[1]) <= input_pos:
                            abs_pos += cigar[1]
                            rel_pos += cigar[1]
                            remain_distance -= cigar[1]
                        else:
                            rel_pos += remain_distance
                            abs_pos += remain_distance
                            remain_distance = 0
                            target = 'no_var'
                            break
                    elif cigar[0] == 4 or cigar[0] == 5:
                        rel_pos += remain_distance
                        abs_pos += remain_distance
                        remain_distance = 0
                        target = 'no_var'
                        break
                    else:
                        print 'cigar[0]>5, unexpected'
                        print read
                        break
                        
                if target == 'var':
                    nvar += 1
                    temp_seq = read.query_sequence[rel_pos:(rel_pos + initial_rlen + ref_rc*call_rlen)]
                    
                    if temp_seq in var_seq:
                        continue
                    else:
                        var_seq.append(temp_seq)
                        
                elif target == 'no_var':
                    nref += 1
                    
                    for ij in range(0,initial_rc+1):
                        temp_seq1.append(read.query_sequence[(rel_pos - (initial_rc-ij)*call_rlen):(rel_pos + ref_rc*call_rlen + ij*call_rlen)])
                    temp_seq2.append(temp_seq1)

                else:
                    print input_line
                    print 'unexpected target'
                    print read
                    sys.exit(1)
                
                
            else:
                nother += 1
                #print 'no target read'
    #for ii in ref_seq1:
        #if ii in var_seq:
            #ndup += 1
        #else:
            #if ref_seq2[ref_seq1.index(ii)] in var_seq:
                #ndup +=1
            #else:
                #'blank'
    #nvar += ndup
    #nref -= ndup
    #print temp_num
    return [nvar,nref,nother,var_seq,temp_seq2]


    
#input_fn = sys.argv[1]    
input_file = file('S1-0Gy-1-all.strelka_varscan_intersection_withVaf.vcf')
output_file = file('S1-0Gy-1-all.strelka_varscan_intersection_withVaf_trial.vcf','w')
t_bam = pysam.AlignmentFile("/home/users/jhyouk/06_mm10_SNUH_radiation/03_bam/S1-0Gy-1.s.md.ir.br.rg.bam",'rb')
n_bam = pysam.AlignmentFile("/home/users/jhyouk/06_mm10_SNUH_radiation/03_bam/N1spleen.s.md.ir.br.rg.bam",'rb')
ref_fasta = pysam.FastaFile("/home/users/jhyouk/99_reference/mouse/mm10/GRCm38.fa")

input_line = input_file.readline().strip()
while input_line[0:1] == '#':
    output_file.write(input_line + '\t' + 't_var;t_ref;t_other;t_dup;n_var;n_ref;n_other;n_dup;Vaf' + '\n')
    input_line = input_file.readline().strip()
#input_line = '17\t19189972\t.\tT\tTAG'
#input_line = '3\t81870660\t.\tT\tTTTC'
#input_line = '13\t77020483\t.\tT\tTTTTCTTTC'

while input_line:
    tdup = 0
    ndup = 0
    info = ''
    if len(input_line.split('\t')[3]) < len(input_line.split('\t')[4]):
        var_list = insertion_bam(input_line,t_bam,ref_fasta)
        #print var_list
        #ref_list = insertion_bam(input_line,n_bam,ref_fasta)
        #print ref_list
        for j in ref_list[4]:
            for m in j:
                if m in var_list[3]:
                    #print m
                    ndup+=1
                    break

        for jj in var_list[4]:
            for mm in jj:
                if mm in var_list[3]:
                    tdup+=1
                    break
        info = str(var_list[0]+tdup) + ';' + str(var_list[1]-tdup) + ';' + str(var_list[2]) +';'+ str(tdup)+';'+ str(ref_list[0]+ndup) +';'+ str(ref_list[1]-ndup) +';'+ str(ref_list[2]) +';'+ str(ndup)+';'+str(round(float(var_list[0]+tdup)/float(var_list[0]+var_list[1]),2))
        output_file.write(input_line +'\t' + info + '\n')
        
    elif len(input_line.split('\t')[3]) > len(input_line.split('\t')[4]):
        'blank'
    else:
        print 'call SNP'
        
    input_line = input_file.readline().strip()

#print 'end'
#close(t_bam)
#close(ref_fasta)